In [19]:
import pandas as pd
from pathlib import Path
import json
import jieba
results = []

for fp in Path("/Users/wanghuan/Projects/simple-stt/outputs-aishell1").glob("*.json"):
    with open(fp, "r") as f:
        d = json.load(f)
        
    text = "".join([r["text"] for r in d])
    results.append([fp.stem, text])
    
actuals = pd.read_feather("/Users/wanghuan/Projects/simple-stt/aishell/transcriptions.fth")
actuals['text'] = actuals['text'].str.replace(" ", "")
actuals.rename(columns = {'text':'target'}, inplace = True)
    
preds = pd.DataFrame(results, columns = ['fname','pred'])
merged = actuals.merge(preds, on='fname', how='inner')

In [20]:
def remove_punctuation(text):
    text = str(text)
    import re
    return re.sub(r'[^\w\s]', '', text)

def jieba_cut(text):
    text = str(text)
    return " ".join(jieba.cut(text))

merged['pred'] = merged['pred'].apply(remove_punctuation)
merged['target'] = merged['target'].apply(remove_punctuation)
merged['pred-cut'] = merged['pred'].apply(jieba_cut)
merged['target-cut'] = merged['target'].apply(jieba_cut)


In [21]:
import jiwer
import numpy as np
wers = []
cers = []
for i, row in merged.iterrows():
    wer = jiwer.wer(row['pred-cut'], row['target-cut'])
    wers.append(wer)
    
    cer = jiwer.cer(row['pred'], row['target'])
    cers.append(cer)
    

merged['wer'] = wers
merged['cer'] = cers
print(f"Average WER: {np.mean(wers)}")
print(f"Average CER: {np.mean(cers)}")

Average WER: 0.05195813255372078
Average CER: 0.026795684767837814


In [12]:
merged

,fname,target,pred,pred-cut,target-cut,wer,cer
0,BAC009S0901W0146,本轮住房公积金房贷政策调整\n,本轮住房公积金房贷政策调整,本轮 住房 公积金 房贷 政策 调整,本轮 住房 公积金 房贷 政策 调整 \n,0.000000,0.000000
1,BAC009S0901W0152,其中刚需买家入市积极性明显提高\n,其中刚需买家入市积极性明显提高,其中 刚需 买家 入市 积极性 明显提高,其中 刚需 买家 入市 积极性 明显提高 \n,0.000000,0.000000
2,BAC009S0901W0185,现在公积金贷款办理需要一个月左右的时间\n,现在公积金贷款办理需要一个月左右的时间,现在 公积金 贷款 办理 需要 一个月 左右 的 时间,现在 公积金 贷款 办理 需要 一个月 左右 的 时间 \n,0.000000,0.000000
3,BAC009S0901W0191,信息化水平不断提高\n,信息化水平不断提高,信息化 水平 不断 提高,信息化 水平 不断 提高 \n,0.000000,0.000000
4,BAC009S0901W0218,指导思想基本原则与发展目标\n,指导思想基本原则与发展目标,指导思想 基本 原则 与 发展 目标,指导思想 基本 原则 与 发展 目标 \n,0.000000,0.000000
...,...,...,...,...,...,...,...
195,BAC009S0916W0419,小笼包身体有反应\n,小笼包身体有反应,小笼包 身体 有 反应,小笼包 身体 有 反应 \n,0.000000,0.000000
196,BAC009S0916W0424,陈妍希晒出与容祖儿合照\n,陈妍希晒出与容祖儿合照,陈妍 希晒出 与 容祖儿 合照,陈妍 希晒出 与 容祖儿 合照 \n,0.000000,0.000000
197,BAC009S0916W0425,并送上真挚祝福\n,并送上真挚祝福,并 送 上 真挚 祝福,并 送 上 真挚 祝福 \n,0.000000,0.000000
198,BAC009S0916W0430,获得粉丝祝福\n,获得粉丝祝福,获得 粉丝 祝福,获得 粉丝 祝福 \n,0.000000,0.000000


In [42]:
from IPython.display import Audio
import soundfile
Audio("../aishell/BAC009S0901W0350.wav",autoplay=True,rate=16000)